In [1]:
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
import seaborn as sns
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.datasets import make_blobs
from sklearn.inspection import DecisionBoundaryDisplay
from keras.utils import to_categorical
import pickle
import tensorflow as tf


%matplotlib inline
import warnings


# Description of the keras function used here


## Flatten

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten

Converets a tensor (think of a multi-dimentional array) to one dimention.

**Example**

A tensor $[[1,2], [3,4]]$ passed trough Flatten will be converted to $[1,2,3,4]$

## Conv2D

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D

Creates a convolution layer that connects the layers above and before it.

## MaxPooling2D

https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D


# Load and Prepare the data

To keep things simpler we will load the data directly from the 
keras repo which comes automatically split to testing and training data as we can see here:

In [18]:
mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the pixel values..
x_train = x_train/255.0
x_test = x_test/255.0


Note that the the features both for training and testing are represented as a 2 dimentional  $28X28$ matrix 

In [19]:
x_train

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

The labels are represented as a one dimentional array of integeres ranging from 0 to 9 (each one corresponding to the $28X28$ pixel matrix that we have seen above):

In [20]:
y_train

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

At this point we will need to perform yet another transformation to covert the features
to "tensor" friendly format as we can see here:

In [23]:
x_train = x_train.reshape(60000, 28, 28, 1)
x_train

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [24]:
x_test = x_test.reshape(10000, 28, 28, 1)
x_test

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

Great! Now we have both the training and the testing features (the $28X28$ matrix of pixel values)
in a format that is compatible with the way that tensorflow needs them and we are ready to train
our model.

# Build the CNN model

In [26]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2https://www.tensorflow.org/api_docs/python/tf/keras/layers/FlattenD(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_5 (Flatten)         (None, 1600)              0         
                                                                 
 dense_8 (Dense)             (None, 128)              

# Train the CNN Model

In [27]:
model.fit(x_train, y_train, epochs=25)

Epoch 1/25


2022-10-31 22:02:14.957480: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500
2022-10-31 22:02:16.543837: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1875/1875 [==============================] - 13s 5ms/step - loss: 0.4340 - accuracy: 0.8415
Epoch 2/25
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2912 - accuracy: 0.8936
Epoch 3/25
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2445 - accuracy: 0.9088
Epoch 4/25
1875/1875 [==============================] - 10s 6ms/step - loss: 0.2141 - accuracy: 0.9200
Epoch 5/25
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1862 - accuracy: 0.9309
Epoch 6/25
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1650 - accuracy: 0.9380
Epoch 7/25
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1443 - accuracy: 0.9461
Epoch 8/25
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1291 - accuracy: 0.9507
Epoch 9/25
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1119 - accuracy: 0.9575
Epoch 10/25
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1001 

In [28]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print ('Test loss: {}, Test accuracy: {}'.format(test_loss, test_accuracy*100))

313/313 [==============================] - 1s 4ms/step - loss: 0.5924 - accuracy: 0.9092
Test loss: 0.5923870205879211, Test accuracy: 90.92000126838684
